In [1]:
import cv2
import numpy as np 
from skimage.segmentation import slic, mark_boundaries
from skimage.util import img_as_float
from matplotlib import pyplot as plt

In [56]:
img_path = '0572.png'

img = cv2.imread(img_path)
b,g,r = cv2.split(img)
B = np.array(b)
R = np.array(r)
row, col = img.shape[0], img.shape[1]

In [135]:
# 阈值法 
# R/B > 0.6 为云，否则为天空
def method_1(thresh=0.6):
    tmp = R / B
    for i in range(row):
        for j in range(col):
            if tmp[i][j] > thresh:
                tmp[i][j] = 255
            else:
                tmp[i][j] = 0
    return tmp        

In [136]:
seg_img = method_1(0.6)
name = img_path.split('.')[0] + '_Ratio_0.6' + '.png'
cv2.imwrite(name, seg_img)

True

In [59]:
# 阈值法 
# B/R < 1.3 为云，否则为天空
def method_2(thresh=1.2):
    tmp = B / R
    for i in range(row):
        for j in range(col):
            if tmp[i][j] < thresh:
                tmp[i][j] = 255
            else:
                tmp[i][j] = 0
    return tmp      

In [60]:
seg_img = method_2(1.2)
name = img_path.split('.')[0] + '_Ratio_1.2' + '.png'
cv2.imwrite(name, seg_img)

True

In [139]:
# 阈值法
# B-R > 30 为天空， 否则为云
def method_3(thresh=30):
    v = np.empty((row,col))
    tmp = B-R
    tmp = tmp.astype(np.int8)
    for i in range(row):
        for j in range(col):
            if tmp[i][j] > thresh:
                v[i][j] = 0
            else:
                v[i][j] = 255
    return v

In [140]:
seg_img = method_3(30)
name = img_path.split('.')[0] + '_Diff' + '.png'
cv2.imwrite(name, seg_img)

True

In [141]:
# 自适应阈值法
# 根据R-B的结果用Ostu法选择阈值
def method_4():
    tmp = R-B
    tmp.dtype=np.int8
    tmp = tmp.astype(np.int16)
    tmp = tmp-tmp.min()
    tmp = tmp.astype(np.uint8)
    thres, res = cv2.threshold(tmp,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    return res

In [142]:
seg_img = method_4()
name = img_path.split('.')[0] + '_Adap' + '.png'
cv2.imwrite(name, seg_img)

True

In [143]:
# 自动图割法 AGC
def method_5():
    tmp = R / B
    x = (tmp-tmp.min())/(tmp.max()-tmp.min())*255
    x = np.round(x)
    x = x.astype(np.uint8)
    T, res = cv2.threshold(x,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    theta = 1.3
    hand_constraint = np.zeros((x.shape[0],x.shape[1]), np.uint8)
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            if x[i][j] > theta * T:
                hand_constraint[i][j] = 1
            elif x[i][j] < 1 / theta * T:
                hand_constraint[i][j] = 0
            else:
                hand_constraint[i][j] = 2
    bgdModel = np.zeros((1,65), np.float64)
    fgdModel = np.zeros((1,65), np.float64)
    mask, bgdModel, fgdModel = cv2.grabCut(img, hand_constraint, None, bgdModel, fgdModel, iterCount=5, mode=cv2.GC_INIT_WITH_MASK )
    mask = np.where((mask==1),1,0).astype('uint8')
    mask = mask*255
    return mask

In [144]:
seg_img = method_5()
name = img_path.split('.')[0] + '_AGC' + '.png'
cv2.imwrite(name, seg_img)

True

In [28]:
# SLIC + B/R
def method_6():
    segments = slic(img_as_float(img), n_segments = 300, sigma = 5)
    img_float = img_as_float(img)
    b, g, r = cv2.split(img_float)
    row, col = img.shape[0], img.shape[1]
    res = np.zeros((row,col))
    sum_r_list = np.zeros((np.max(segments)+1, 1))
    sum_b_list = np.zeros((np.max(segments)+1, 1))
    for i in range(row):
        for j in range(col):
            label = segments[i][j]
            sum_r_list[label] += r[i][j]
            sum_b_list[label] += b[i][j]
    r_b_list = sum_b_list / sum_r_list
    for i in range(row):
        for j in range(col):
            label = segments[i][j]
            if r_b_list[label] < 1.2:
                res[i][j] = 255
            else:
                res[i][j] = 0
    return res

In [29]:
seg_img = method_6()
name = img_path.split('.')[0] + '_SLIC_1.2' + '.png'
cv2.imwrite(name, seg_img)

True